# Project_One_이상봉

## 주제와 목적
최근 북미, 유럽, 일본, 그외 지역에서의 게임 판매량 데이터를 분석하고  
분석을 바탕으로 다음 분기에 게임 설계 방향을 정하는것이다    
정확한 분석을 위해서는 게임의 판매량에 영향을 미치는요인이 무엇인지 알아내야한다.  

## 우선 분석과정을 4개의 작은 단위로 나눠볼 수 있다
1. 지역별 선호하는 장르가 다른가?
2. 연도별 트랜드가 있는가?
3. 플랫폼에따른 판매량의 차이가 있는가?
4. 판매량이 가장 높은 게임들은 무슨 특징을 가지고 있는가?

이 분석과정을 통해, **"잘 팔리는"** 게임을 만들기위해 어떤 부분을 우선시 해야하는지를 판단해야 한다


분석을 시작하기전에, 데이터를 좀더 깔끔하게 다듬는 전처리 작업을 진행하였다  
# 전처리작업(EDA)
우선 데이터내의 수치를 일관성있게 통일하기위해 전처리작업을 진행하였다  
전처리작업이란, 숫자의단위나 알파벳의 대소문자의 통일, 알수없는값이나 비어있는값을 처리하여 데이터의 오류를 줄이는 과정이다.

In [104]:
import pandas as pd


# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/AI_bootcamp/vgames2.csv', encoding='CP949')

# 인덱스제거
df = df.drop(['Unnamed: 0','Name','Publisher'], axis=1)

In [105]:
# 판매량을 (0.00) 단위로 통일하고 float으로 변경
def unification(string):
    # K가 있다면 지우고 천 곱하고 백만으로 나누기
    if ('K' in string) == True:
        # 만약 0K 라면 0.001로 바로바꾸기
        if string == '0K':
            return float(0.001)
        else:
            remove_k = string.replace('K','')
            return float(remove_k)*0.001
    # M이 있다면 지우기
    elif ('M' in string) == True:
        # 만약 OM 이라면 1.0으로 바꾸기
        if string == '0M':
            return float(1.0)
        else:
            remove_m = string.replace('M','')
            return float(remove_m)
    # 둘다없다면 타입만바꾸기
    else:
        return float(string)

# 판매량 칼럽의 단위통일시키기
df['NA_Sales'] = df['NA_Sales'].apply(unification)
df['JP_Sales'] = df['JP_Sales'].apply(unification)
df['EU_Sales'] = df['EU_Sales'].apply(unification)
df['Other_Sales'] = df['Other_Sales'].apply(unification)

# 480K 값이 0.48로 잘 변경되었는지 확인
df['NA_Sales'][10]

# datatype이 object->float으로 되었는지 확인
df.dtypes

Platform        object
Year           float64
Genre           object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
dtype: object

In [106]:
# Year 칼럼 네자리로 통일시키기
def make_year(year):
    if year < 21:
        return float(year+2000)
    elif year > 80 and year <= 99:
        return float(year+1900)
    else:
        return float(year)

df['Year'] = df['Year'].apply(make_year)

In [107]:
# 결측치 위치 확인하기
# 이값은 후에 다시 다루기
df.isna().sum()

Platform         0
Year           271
Genre           50
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

전처리가 끝났으니 데이터를 분석해보자

# 지역별 선호하는 장르가 다른가?  
선호하는 장르란?  "많이 팔리는 장르"이다.  
한마디로 **"지역별 판매량과, 장르가 서로 관련이 있는가?"** 를 확인해야한다  

만약 장르가 두가지 뿐이라면 어떤 장르가 더 잘팔리는지를 비교하면 된다
하지만 수많은 장르와 수만개이상의 게임을 비교한다면... 한번에 정확한 비교를 할 수 없다  

그러므로, 통계학적 방법을 이용하여 데이터를 **단순화** 시켜, 효율적이고 정확한 비교를 해야한다  

단순화한다는게 무슨뜻일까?  
예를들면 사람의 외모나 능력, 습관을 가지고 "이사람은 예의바른사람인가?" 를 판단하긴 힘들다.  
하지만 그 사람이 초면에 반말을하는 습관이있다면, 예의없는 사람일 확률이 매우높다.  
즉, 외모나 능력보다 반말을한다는 조건이 비교적 확실한 판단 기준이 될 수 있다.  
동일하게, 데이터의 특징들 중 비교적 확실한 판단기준이되는 특징을 찾고(경우에따라 특징들을 혼합하기도한다) 그것을 기준으로 나누는것이 단순화라고 볼 수 있다.